In [2]:
# End Goal:
#     df = pd.read_csv("data.csv")
#     tree = decision_tree_algorithm(train_df)
#     accuracy = calculate_accuracy(test_df, tree)
#     # ie calculate the decision tree on any dataset and use the test data to calculate accuracy of that training on test.
    

In [3]:
# Import statements
import numpy as np
import pandas as pd

#Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Makes sure that the plots are displayed in notebook
%matplotlib inline

import random
from pprint import pprint

In [4]:
#Load and Prepare Data
df = pd.read_csv("iris.csv")
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
#Last column of the data frame must contain the label and it must also be called label
df=df.rename(columns={"Species":"label"})
df=df.rename(columns={"Petal.Length":"petal_length"})
df=df.rename(columns={"Petal.Width":"petal_width"})
df=df.rename(columns={"Sepal.Width":"sepal_width"})
df=df.rename(columns={"Sepal.Length":"sepal_length"})

df.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
#There should be no missing values in dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
label           150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [6]:
#Train-Test Split
def train_test_split(df, test_size):
    if isinstance(test_size,float):
        test_size =round(test_size*len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df= df.drop(test_indices)
    return train_df, test_df

In [7]:
random.seed(0)

train_df, test_df = train_test_split(df,test_size=20)

In [8]:
 #Getting a numpy 2D array
data = train_df.values
data[:5]

array([[5.1, 3.5, 1.4, 0.2, 'setosa'],
       [4.9, 3.0, 1.4, 0.2, 'setosa'],
       [4.7, 3.2, 1.3, 0.2, 'setosa'],
       [4.6, 3.1, 1.5, 0.2, 'setosa'],
       [5.0, 3.6, 1.4, 0.2, 'setosa']], dtype=object)

In [9]:
### Data pure?
def check_purity(data):
    label_column = data[ : , -1]
    unique_classes = np.unique(label_column)

    if(len(unique_classes) == 1):
        return True
    else:
        return False

In [10]:
###Classify
def classify_data(data):
    label_column = data[ : , -1]
    unique_classes,counts_unique_classes = np.unique(label_column, return_counts=True)
    index  = counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification

In [47]:
### Potential Splits
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns-1):
        values = data[:,column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index-1]
                    potential_split = (current_value+previous_value) / 2
            
                    potential_splits[column_index].append(potential_split)
        else:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [48]:
get_potential_splits(train_df.values)

{0: array([1, 2, 3], dtype=object),
 1: array(['female', 'male'], dtype=object),
 2: [0.71,
  0.79,
  0.875,
  0.96,
  1.5,
  2.5,
  3.5,
  4.5,
  5.5,
  6.5,
  7.5,
  8.5,
  9.5,
  10.5,
  11.5,
  12.5,
  13.5,
  14.5,
  15.5,
  16.5,
  17.5,
  18.5,
  19.5,
  20.5,
  21.5,
  22.5,
  23.25,
  23.75,
  24.5,
  25.5,
  26.5,
  27.5,
  28.25,
  28.75,
  29.5,
  30.25,
  30.75,
  31.5,
  32.25,
  32.75,
  33.5,
  34.5,
  35.5,
  36.25,
  36.75,
  37.5,
  38.5,
  39.5,
  40.25,
  40.75,
  41.5,
  42.5,
  43.5,
  44.5,
  45.25,
  45.75,
  46.5,
  47.5,
  48.5,
  49.5,
  50.5,
  51.5,
  52.5,
  53.5,
  54.5,
  55.25,
  55.75,
  56.5,
  57.5,
  58.5,
  59.5,
  60.5,
  61.5,
  62.5,
  63.5,
  64.5,
  67.5,
  70.25,
  70.75,
  72.5,
  77.0],
 3: array([0, 1, 2, 3, 4, 5, 8], dtype=object),
 4: array([0, 1, 2, 3, 4, 5, 6], dtype=object),
 5: [2.00625,
  4.50625,
  5.61875,
  6.36665,
  6.6229,
  6.80415,
  6.91665,
  7.0104,
  7.0479,
  7.052099999999999,
  7.0896,
  7.175,
  7.2271,
  7.23959999

In [50]:
#Split Data
def split_data(data, split_column, split_value):
    split_column_values = data[:,split_column]
    
    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values > split_value]
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    return data_below,data_above

In [44]:
data_below,data_above = split_data(train_df.values,split_column = 1, split_value ="male")

In [46]:
np.unique(data_above[:,1])

array(['female'], dtype=object)

In [51]:
### Lowest Overall Entropy:
def calculate_entropy(data):
    label_column = data[:,-1]
    _,counts = np.unique(label_column, return_counts=True)

    probabilities = counts/counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [52]:
def calculate_overall_entropy(data_below,data_above):
    n_data_points = len(data_below)+len(data_above)

    p_data_below = len(data_below)/n_data_points
    p_data_above = len(data_above)/n_data_points

    overall_entropy = (p_data_below * calculate_entropy(data_below)
                       +p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [53]:
def determine_best_split(data,potential_splits):
    overall_entropy = 999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below,data_above = split_data(data, split_column = column_index, split_value = value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
        
            if(current_overall_entropy <= overall_entropy):
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    return best_split_column, best_split_value

In [54]:
## Determine Type of Feature

def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_threshold = 15
    
    for column in df.columns:
        unique_values = df[column].unique()
        example_value = unique_values[0]
        
        if(isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_threshold):
            feature_types.append("categorical")
        else:
            feature_types.append("continuous")
    
    return feature_types

In [55]:
FEATURE_TYPES

['categorical',
 'categorical',
 'continuous',
 'categorical',
 'categorical',
 'continuous',
 'categorical',
 'categorical']

In [56]:
## Algorithm: recursive and uses dataframe because we have to use it as api
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth = 5):
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df
    
    #Base Cases
    if (check_purity(data)) or (len(data) < min_samples) or(counter == max_depth):
        classification = classify_data(data)
        return classification
    
    #Recursive Part
    else:
        counter +=1
        
        #Helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data,split_column,split_value)
        
        #Instantiate Subtree
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name,split_value)
        else:
            question = "{} = {}".format(feature_name,split_value)
        sub_tree = {question: []}
        
        #find answers(recursive part)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples,max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples,max_depth)
        
        if(yes_answer == no_answer):
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [58]:
tree = decision_tree_algorithm(train_df,max_depth=3)
pprint(tree,width=50)

{'Sex = male': [{'Fare <= 10.81665': [0,
                                      {'Age <= 6.5': [1,
                                                      0]}]},
                {'Pclass = 3': [{'Fare <= 23.7': [1,
                                                  0]},
                                1]}]}


In [62]:
example = test_df.iloc[0]
example

Pclass           3
Sex           male
Age             28
SibSp            0
Parch            0
Fare        7.8958
Embarked         S
label            0
Name: 101, dtype: object

In [60]:
## Classification
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    #Ask Question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    else:
        if str(example[feature_name])== value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    #Base Case
    if not isinstance(answer,dict):
        return answer

    #Recursive part
    else:
        residual_tree = answer
        return classify_example(example,residual_tree)

In [24]:
classify_example(example,tree)

'virginica'

In [25]:
##Accuracy 
def calculate_accuracy(df,tree):
    
    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df.classification == df.label
    
    accuracy = df.classification_correct.mean()
    return accuracy

In [26]:
calculate_accuracy(test_df, tree)

0.95

In [27]:
# Titanic Data set
#Prepare and Load data
df = pd.read_csv("Titanic.csv")
df["label"] = df.Survived
df = df.drop(["PassengerId","Survived","Name","Ticket","Cabin"], axis=1)


##Handling Missing values
median_age = df.Age.median()
mode_embarked = df.Embarked.mode()[0]

df = df.fillna({"Age": median_age, "Embarked":mode_embarked})

In [28]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,label
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [65]:
## Decision Tree Algorithm
train_df, test_df = train_test_split(df, test_size=0.2)
tree = decision_tree_algorithm(train_df,max_depth =3)
accuracy = calculate_accuracy(test_df, tree)
pprint(tree, width =50)
print(accuracy)

{'Sex = male': [{'Fare <= 26.26875': [{'Age <= 13.5': [1,
                                                       0]},
                                      0]},
                {'Pclass = 3': [{'Fare <= 23.35': [1,
                                                   0]},
                                {'Age <= 2.5': [0,
                                                1]}]}]}
0.8202247191011236


In [37]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,label
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [38]:
feature_types = determine_type_of_feature(df)
i=0

for column in df.columns:
    print(column,"-",feature_types[i])
    i+=1

Pclass - categorical
Sex - categorical
Age - continuous
SibSp - categorical
Parch - categorical
Fare - continuous
Embarked - categorical
label - categorical
